Подключаемые библиотеки

In [1]:
import re
import random
import urllib2
import math
import numpy as np
import scipy.interpolate as ip
import pandas as pd
import pandas.io.data as web
import matplotlib.pyplot as plt
import scipy.optimize as sco

from bs4 import BeautifulSoup

/usr/lib64/python2.7/site-packages/pandas/io/data.py:33: FutureWarning: 
The pandas.io.data module is moved to a separate package (pandas-datareader) and will be removed from pandas in a future version.
After installing the pandas-datareader package (https://github.com/pydata/pandas-datareader), you can change the import ``from pandas.io import data, wb`` to ``from pandas_datareader import data, wb``.
  FutureWarning)


Настройки отображения графиков:

In [2]:
%matplotlib inline
plt.style.use('fivethirtyeight') # The most fanciest styles are 'ggplot' and 'fivethirtyeight'
# All available styles may be checked by
#print plt.style.available

In [3]:
class sp_500():
    sector_tickers = {}
    symbols = []
    
    def __init__(self):
        self.symbols = []
        self.sector_tickers = dict()
        
        site = "http://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
        hdr = {'User-Agent': 'Mozilla/5.0'}
        req = urllib2.Request(site, headers=hdr)
        page = urllib2.urlopen(req)
        soup = BeautifulSoup(page)

        table = soup.find('table', {'class': 'wikitable sortable'})
    
        for row in table.findAll('tr'):
            col = row.findAll('td')
            if len(col) > 0:
                sector = str(col[3].string.strip()).lower().replace(' ', '_')
                ticker = str(col[0].string.strip())
                date = str(col[6]).replace('<td>', '').replace('</td>', '')
                #if date < historical_end_date:
                if sector not in self.sector_tickers:
                    self.sector_tickers[sector] = list()
                self.sector_tickers[sector].append(ticker)
                self.symbols.append(ticker)
        
       
    # Show sectors summary
    def sectors(self):
        for s in self.sector_tickers:
            print s + ': ' + str(len(self.sector_tickers[s])) + ' companies'
            
    # Show the sector summary and companies in it
    def show(self, sector = None):
        if sector == None:
            self.sectors()
        else:
            print sector + ' sector of S&P500\n' + str(len(self.sector_tickers[sector])) + ' companies:'
            print self.sector_tickers[sector]
    
    # Get the company symbols
    def get(self, sector = None):
        symbols = []
        if sector == None:
            # Get the companies from all S&P500 sectors
            for s in self.sector_tickers:
                print s + ': ' + str(len(self.sector_tickers[s])) + ' companies'
        else:
            # Get the companies from the given sector
            for sym in self.sector_tickers[sector]:
                symbols.append(sym)
        return symbols
    
    # Get random number of stocks
    def random(self, num):
        return random.sample(self.symbols, num)

In [4]:
class portfolio:
    data = []
    returns = []
    start_prices = []
    end_prices = []
    
    min_return = 0.0 # minimal return value 
    max_return = 0.7 # maximum return value
    step = 100 
    threshold = 0.001 # the companies with weights below this value are not included in portfolio
    
    # Constructor
    def __init__(self, _symbols, _start, _end, _risk, _weights = None):
        # Initializing data
        self.data = pd.DataFrame()
        self.num_of_days = 0
        self.feasible_returns = []
        self.feasible_volatilities = []
        self.feasible_weights = []
    
        self.effective_volatilities = []
        self.effective_returns = []
        self.effective_weights = []
        
        self.symbols = []
        self.sharpe_symbols = []
        self.min_variance_symbols = []
        self.days = []
        
        # 
        self.symbols = _symbols
        self.start = _start
        self.end = _end
        self.max_risk = _risk
        
        self.fetch()
        self.set_returns() 
        self.set_start_end_prices()
        
        if _weights != None:
            self.test(_weights)
    
    # Get percent-view of an number (-24.5% instead of -0.245)
    def percent(self, i):
        formatted = str(i.round(3) * 100) + '%'
        if i > 0:
            formatted = '+' + formatted
        return formatted
    
    #
    def set_days(self):
        for sym in self.symbols:
            self.days = []
            for i in range(0, self.num_of_days):
                self.days.append( str( re.split(' ', str(self.data[sym].axes[0][i]))[0]) )
                
            if len(self.days) == self.num_of_days:
                break
                
        if self.start != self.days[0] or self.end != self.days[-1]:
            self.start = self.days[0]
            self.end = self.days[-1]
            print 'Investment period adjusted to ' + self.start + ' - ' + self.end
    
    # Вычисление логарифмически нормализованых доходностей (отношения уровня закрытия текущего дня к предыдущему) за исторический период   
    def set_returns(self):
        self.returns = np.log(self.data / self.data.shift(1))
    
    # 
    def set_start_end_prices(self):
        self.start_prices = []
        self.end_prices = []
        for sym in self.symbols:
            self.start_prices.append(self.data[sym][self.start])
            self.end_prices.append(self.data[sym][self.end])
    
    # Retrieving data
    def fetch(self):
        fetched_symbols = []
        for sym in self.symbols:
            try:
                self.data[sym]  = web.DataReader(sym, data_source='yahoo', start=self.start, end=self.end)['Adj Close']
                fetched_symbols.append(sym)           
                if self.data[sym].size > self.num_of_days:
                    self.num_of_days = self.data[sym].size
            except IOError:
                print sym + ': no data available for requested period'
                continue
                
        self.data.column = fetched_symbols
    
        for sym in self.symbols:
            if sym not in fetched_symbols:
                self.symbols.remove(sym)
                
        self.set_days()
        print 'Data fetched for ' + str(len(self.symbols)) + ' symbols and '+ str(len(self.days)) + ' trading days'
    
    #
    def show(self, weights):
        print "Investment period: " + self.start + " - " + self.end
        print "Risk limit: " + str(self.max_risk)
        
        print "Portfolio volatility: " + str( self.statistics(weights)[0].round(3) )
        print "Portfolio return: " + str( self.statistics(weights)[1].round(3) )   
        print "Sharpe ratio: "  + str( self.statistics(weights)[2].round(3) )  
        
        print "Portfolio weights:"    
        for sym, w in zip(self.symbols, weights):
            if w > self.threshold:
                print sym + ': ' + str(w.round(3))
    
    # Show the line with symbol statistics
    # AAPL: 67.802 -> 72.73 (+7.3%)
    def show_symbol(self, sym, start_price, end_price, return_rate):
        print sym + ': ' + str(start_price.round(3)) + ' -> ' + str(end_price.round(3)) + ' (' + self.percent(return_rate) + ')'
    
    #
    def test(self, weights):     
        portfolio_start = 0
        portfolio_end = 0
        
        for s, sp, ep, w in zip(self.symbols, self.start_prices, self.end_prices, weights):
            ret = (ep - sp) / sp  
            portfolio_start = portfolio_start + sp * w
            portfolio_end = portfolio_end + ep * w
            self.show_symbol(s, sp, ep, ret)
            
        portfolio_return = (portfolio_end - portfolio_start) / portfolio_start
        print '\nPortfolio return: ' + self.percent(portfolio_return)
        
        return
    
    #
    def calc(self):
        self.get_effective_set()
        print 'The highest Sharpe ratio portfolio: '
        self.show( self.get_max_sharpe_weights() )
        
        print '\nThe minimal variance porfolio: '
        self.show( self.get_min_variance_weights() )
        
    
    # 
    def get_risky(self):
        if len(self.sharpe_symbols) > 0:
            return self.sharpe_symbols, self.sharpe_weights
        else:
            print 'The optimal portfolio is not calculated. Run calc() first'
            
    #
    def get_conservative(self):
        if len(self.min_variance_symbols) > 0:
            return self.min_variance_symbols, self.min_variance_weights
        else:
            print 'The optimal portfolio is not calculated. Run calc() first'
        return
    
    # Вычисление эффективного множества (effective set)
    def get_effective_set(self):
        self.effective_volatilities = []
        self.effective_returns = []
        self.effective_weights = []
    
        cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
        bnds = tuple((0, 1) for x in range(len(self.symbols)))

        for y in np.linspace(self.min_return, self.max_return, self.step):
            cons = ({'type': 'eq', 'fun': lambda x: self.statistics(x)[1] - y}, {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
            result = sco.minimize(self.min_volatility, len(self.symbols) * [1. / len(self.symbols),], method='SLSQP', bounds=bnds, constraints=cons)
            if result['fun'] <= self.max_risk:
                self.effective_volatilities.append(result['fun'])
                self.effective_returns.append(y)
                self.effective_weights.append(result['x'])
    
        self.effective_volatilities = np.array(self.effective_volatilities)
        self.effective_returns = np.array(self.effective_returns)
        self.effective_weights = np.array(self.effective_weights)

    # Функция, получающая веса бумаг в портфеле в качестве входных параметров, и возвращающая массив 
    # данных о портфеле в формате [волатильность, доходность, коэффициент Шарпа]
    def statistics(self, weights):
        weights = np.array(weights)
        portfolio_return = self.get_portfolio_return(weights)
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(self.returns.cov() * self.num_of_days, weights)))
    
        return np.array([portfolio_volatility, portfolio_return, portfolio_return / portfolio_volatility])
    
    # Функция вычисления минимального отклонения
    def min_volatility(self, weights):
        return self.statistics(weights)[0]
    
    # Функция вычисления доходности портфеля
    def get_portfolio_return(self, weights):
        portfolio_start_price = 0
        portfolio_end_price = 0

        for w, sp, ep in zip(weights, self.start_prices, self.end_prices):
            portfolio_start_price = portfolio_start_price + w * sp
            portfolio_end_price = portfolio_end_price + w * ep
            
        return ((portfolio_end_price - portfolio_start_price) / portfolio_start_price)
        #return np.sum(returns.mean() * weights) * num_of_days
    
    # Функция вычисления портфеля с максимальным коэффицентом Шарпа (отношением доходность/волатильность)
    def get_max_sharpe_weights(self):
        max_sharpe = 0
        weights = []
        for r, v, w in zip(self.effective_returns, self.effective_volatilities, self.effective_weights):
            if r / v > max_sharpe:
                max_sharpe =  r / v
                weights = w
                
        self.sharpe_symbols = []
        self.sharpe_weights = []
        for sym, w in zip(self.symbols, weights):
            if w > self.threshold:
                self.sharpe_symbols.append(sym)
                self.sharpe_weights.append(w.round(3)) # FIXME: shoudn't round value maybe
        
        return weights
    
    #
    def get_min_variance_weights(self):
        min_variance = 10
        weights = []
        for v, w in zip(self.effective_volatilities, self.effective_weights):
            if v < min_variance:
                min_variance =  v
                weights = w
                
        self.min_variance_symbols = []
        self.min_variance_weights = []
        for sym, w in zip(self.symbols, weights):
            if w.round(3) > 0.001:
                self.min_variance_symbols.append(sym)
                self.min_variance_weights.append(w.round(3)) # FIXME: shoudn't round value maybe

        return weights
    
    def plot(self):
        plt.figure(figsize=(12, 6))

        x_sharpe = self.statistics(self.get_max_sharpe_weights())[0]
        y_sharpe = self.statistics(self.get_max_sharpe_weights())[1]

        x_min_vol = self.statistics(self.get_min_variance_weights())[0]
        y_min_vol = self.statistics(self.get_min_variance_weights())[1]

        #plt.scatter(feasible_volatilities, feasible_returns, c = feasible_returns / feasible_volatilities, marker='o')
        plt.scatter(self.effective_volatilities, self.effective_returns, c = self.effective_returns / self.effective_volatilities, marker='x')
        plt.plot(x_sharpe, y_sharpe, 'rs', markersize=8.0) # portfolio with highest Sharpe ratio
        plt.plot(x_min_vol, y_min_vol, 'ys', markersize=8.0) # portfolio with highest Sharpe ratio

        plt.colorbar(label='Sharpe ratio')
        plt.xlabel(r'$\sigma_p$')
        plt.ylabel(r'$\bar{r_p}$')
        

Получение наименования компаний, входящих в S&P500 на данный момент (в момент запуска) 

In [5]:
sp500 = sp_500()

/usr/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [6]:
sp500.sectors()

industrials: 67 companies
energy: 41 companies
consumer_discretionary: 85 companies
utilities: 29 companies
telecommunications_services: 5 companies
materials: 27 companies
health_care: 56 companies
consumer_staples: 38 companies
financials: 89 companies
information_technology: 67 companies


In [7]:
sp500.random(3)

['BF-B', 'UHS', 'EXPD']

In [7]:
sp500.show('industrials')

industrials sector of S&P500
67 companies:
['MMM', 'ADT', 'ALLE', 'AAL', 'AME', 'APH', 'BA', 'CHRW', 'CAT', 'CTAS', 'GLW', 'CSX', 'CMI', 'DHR', 'DE', 'DAL', 'DOV', 'DNB', 'ETN', 'EMR', 'EXPD', 'FAST', 'FDX', 'FLIR', 'FLS', 'FLR', 'GD', 'GE', 'GWW', 'HON', 'ITW', 'IR', 'IRM', 'JEC', 'JBHT', 'KSU', 'LLL', 'LEG', 'LMT', 'MAS', 'NLSN', 'NSC', 'NOC', 'PCAR', 'PH', 'PNR', 'PBI', 'PWR', 'RTN', 'RSG', 'RHI', 'ROK', 'COL', 'ROP', 'R', 'LUV', 'SRCL', 'TXT', 'TYC', 'UNP', 'UAL', 'UPS', 'URI', 'UTX', 'VRSK', 'WM', 'XYL']


In [8]:
max_risk = 0.2
#p = portfolio(sp500.get('telecommunications_services'), '2014-01-01', '2015-01-01', max_risk)
p = portfolio(sp500.random(30), '2014-01-01', '2015-01-01', max_risk)

KHC: no data available
Investment period adjusted to 2014-01-02 - 2014-12-31
Data fetched for 29 symbols and 252 trading days


Вычисление оптимальных портфелей

In [9]:
p.calc()

The highest Sharpe ratio portfolio: 
Investment period: 2014-01-02 - 2014-12-31
Risk limit: 0.2
Portfolio volatility: 0.144
Portfolio return: 0.601
Sharpe ratio: 4.173
Portfolio weights:
FE: 0.249
AVGO: 0.292
RSG: 0.36
AAL: 0.099

The minimal variance porfolio: 
Investment period: 2014-01-02 - 2014-12-31
Risk limit: 0.2
Portfolio volatility: 0.095
Portfolio return: 0.12
Sharpe ratio: 1.263
Portfolio weights:
FE: 0.118
NKE: 0.012
L: 0.039
PM: 0.218
JNPR: 0.063
KORS: 0.009
COG: 0.04
HSIC: 0.169
MDLZ: 0.002
RSG: 0.33


In [ ]:
p.plot()

Оптимальный портфель (портфель с максимальным коэффициентом Шарпа):

In [ ]:
#p.get_risky()

Оптимальный портфель (портфель с наименьшей волатильностью)

In [ ]:
#p.get_conservative()

Проверяем доходности двух полученных оптимальных портфелей на нужном промежутке времени

In [ ]:
r = portfolio(p.get_risky()[0], '2015-01-01', '2016-01-01', None, p.get_risky()[1])

In [ ]:
c = portfolio(p.get_conservative()[0], '2015-01-01', '2016-01-01', None, p.get_conservative()[1])